In [1]:

from ultralytics import YOLO

import torch

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [3]:
model = YOLO("/home/maxim/DS/ds-phase-2/cv_project/yolo8m_model/weights/best.pt")

In [3]:
print(f"Классы модели: {model.names}")

Классы модели: {0: 'Speedlimit', 1: 'Stopsign', 2: 'Trafficlight'}


In [7]:
import json

# Если model.names уже словарь {id: name}
model_classes = model.names

data = {
    "classes": model_classes
}

with open('model_classes.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="sclPIxmqfKhFNRaI78wp")
project = rf.workspace("project-a0jvt").project("segmentation-speedlimit-gqsun")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Segmentation-Speedlimit-2 in yolov8:: 100%|██████████| 488/488 [00:00<00:00, 5320.05it/s]


In [8]:
results = model.predict(
    source='/home/maxim/DS/ds-phase-2/cv_project/Initial dataset/images/road22.png',
    save=True,
    show=False, 
    project='/home/maxim/DS/ds-phase-2/cv_project/yolo8m_model/results',  # основная папка
    name='experiment',           # подпапка
    exist_ok=True                  # перезаписывать если существует
)


image 1/1 /home/maxim/DS/ds-phase-2/cv_project/Initial dataset/images/road22.png: 640x544 1 Stopsign, 581.3ms
Speed: 8.1ms preprocess, 581.3ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 544)
Results saved to /home/maxim/DS/ds-phase-2/cv_project/yolo8m_model/results/experiment


In [ ]:
metrics = model.val(
    data='../../datasets/Ships',  # ваш data.yaml файл
    split='test',                     # используем валидационный набор
    batch=16,                        # размер батча
    imgsz=640,                       # размер изображения
    conf=0.1,                      # порог confidence для детекции
    iou=0.6,                         # порог IoU
    device=DEVICE,                   # или 'cpu'
    save_json=True,                  # сохранить метрики в JSON
    save_hybrid=True,                # сохранить гибридные метрики
    plots=True                       # построить графики
)

print("✅ Валидация завершена!")


WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.221 🚀 Python-3.12.11 torch-2.9.0+cu128 CPU (Intel Core i5-1035G1 1.00GHz)
YOLOv8m-seg summary (fused): 105 layers, 27,224,121 parameters, 0 gradients, 104.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 102.0±27.4 MB/s, size: 40.3 KB)
val: Scanning /home/maxim/DS/ds-phase-2/cv_project/Train dataset/test/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 60/60 139.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.1it/s 1:07<27.8s
                   all         60         68      0.981      0.987      0.992       0.93      0.981      0.987      0.992      0.878
            Speedlimit         35         38          1      0.974      0.986      0.892          1      0.974      0.986      0.821
              Stopsign         13         13      0.943     

In [4]:
results = model("Train dataset/valid/images/road346_png.rf.f13f89c3bf845d025b8835d6c8315f5a.jpg")  # predict on an image

# Access the results
for result in results:
    xy = result.masks.xy  # mask in polygon format
    xyn = result.masks.xyn  # normalized
    masks = result.masks.data  # mask in matrix format (num_objects x H x W)


image 1/1 /home/maxim/DS/projects/Road-Infrastructure-Detection/Train dataset/valid/images/road346_png.rf.f13f89c3bf845d025b8835d6c8315f5a.jpg: 640x640 1 Speedlimit, 822.5ms
Speed: 11.3ms preprocess, 822.5ms inference, 15.8ms postprocess per image at shape (1, 3, 640, 640)


In [8]:
result = results[0]

# Получаем все данные разом
boxes = result.boxes

# Извлекаем классы, уверенность и координаты
class_indices = boxes.cls.int().tolist()
confidences = boxes.conf.tolist()
class_names = [model.names[i] for i in class_indices]

print("Детекции:")
for i, (class_idx, class_name, conf) in enumerate(zip(class_indices, class_names, confidences)):
    print(f"Объект {i+1}: {class_name} (класс {class_idx}), уверенность: {conf:.2f}")

Детекции:
Объект 1: Speedlimit (класс 0), уверенность: 0.95


In [12]:
result.masks

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)
orig_shape: (640, 640)
shape: torch.Size([1, 640, 640])
xy: [array([[182., 326.],
       [182., 437.],
       [329., 437.],
       [329., 326.]], dtype=float32)]
xyn: [array([[0.284375 , 0.509375 ],
       [0.284375 , 0.6828125],
       [0.5140625, 0.6828125],
       [0.5140625, 0.509375 ]], dtype=float32)]